In [1]:
import json
import pandas as pd

In [2]:
data_in = "../../sih-raw-data/geojson/"
data_out = "../data/geojson/"

In [3]:
with open(data_in + r"gadm36_IND_1.json", "r") as f:
    states = json.load(f)
for i in states["features"]:
    v = i["properties"]["NAME_1"].lower()
    if v == "nct of delhi":
        i["id"] = "delhi"
    elif v == "jammu and kashmir":
        i["id"] = "jammu & kashmir"
    elif v == "puducherry":
        i["id"] = "pondicherry"
    elif v == "andaman and nicobar":
        i["id"] = "andaman & nicobar"
    elif v == "dadra and nagar haveli":
        i["id"] = "dadra and nagar hav"
    elif v == "tamil nadu":
        i["id"] = "tamilnadu"
    else:
        i["id"] = v
with open(data_out + r"gadm36_IND_1_id.json", "w") as f:
    json.dump(states, f)

In [4]:
with open(data_in + r"gadm36_IND_2.json", "r") as f:
    districts = json.load(f)
for i in districts["features"]:
    v = i["properties"]["NAME_2"].lower()
    if v == "allahabad":
        i["id"] = "prayagraj"
    elif v == "mumbai city":
        i["id"] = "mumbai"
    elif v == "north and middle andaman":
        i["id"] = "north & middle andaman"
    elif v == "dadra and nagar haveli":
        i["id"] = "dadra & nagar haveli"
    else:
        i["id"] = v
with open(data_out + r"gadm36_IND_2_id.json", "w") as f:
    json.dump(districts, f)

In [5]:
with open(data_in + r"gadm36_IND_0.json", "r") as f:
    india = json.load(f)
for i in india["features"]:
    i["id"] = i["properties"]["NAME_0"].lower()
with open(data_out + r"gadm36_IND_0_id.json", "w") as f:
    json.dump(india, f)

In [6]:
with open(data_in + r"gadm36_IND_3.json", "r") as f:
    blocks = json.load(f)
for i in blocks["features"]:
    v = i["properties"]["NAME_3"].lower()
    if v == "allahabad":
        i["id"] = "prayagraj"
    else:
        i["id"] = v
with open(data_out + r"gadm36_IND_3_id.json", "w") as f:
    json.dump(blocks, f)

## Rough

In [7]:
with open(data_out + r"gadm36_IND_2_id.json", "r") as f:
    districts_json = json.load(f)
with open(data_out + r"gadm36_IND_3_id.json", "r") as f:
    blocks_json = json.load(f)

In [8]:
df = pd.read_parquet(r"../data/comp/gw-block-pre-post.parquet.gzip")

In [9]:
districts = list(set(list(df["district"])))
blocks = list(set(list(df["block"])))

In [10]:
districts_not_found = []
blocks_not_found = []

In [11]:
for i in districts:
    found = False
    for j in districts_json["features"]:
        if j["id"] == i:
            found = True
            break
    if not found:
        districts_not_found.append(i)
df_not_districts = pd.DataFrame({"districts": districts_not_found})

In [12]:
df_not_districts.sort_values("districts", inplace=True)

In [13]:
with open(data_out + r"gadm36_IND_3_id.json", "r") as f:
    blocks_json = json.load(f)

In [14]:
for i in blocks:
    found = False
    for j in blocks_json["features"]:
        if j["id"] == i:
            found = True
            break
    if not found:
        blocks_not_found.append(i)
df_not_blocks = pd.DataFrame({"blocks": blocks_not_found})
df_not_blocks.sort_values("blocks", inplace=True)

In [15]:
df_not_blocks.to_csv(r"../data/blocks_not_found.csv", index=False)

In [16]:
df_not_districts.to_csv(r"../data/districts_not_found.csv", index=False)

In [17]:
block = []
for j in blocks_json["features"]:
    block.append(j["id"])
all_blocks_json = pd.DataFrame({"blocks": block})
all_blocks_json.sort_values("blocks", inplace=True)

In [18]:
all_blocks_json.to_csv(r"../data/all-blocks-json.csv", index=False)

In [19]:
district = []
for j in districts_json["features"]:
    district.append(j["id"])
all_districts_json = pd.DataFrame({"districts": district})
all_districts_json.sort_values("districts", inplace=True)

In [20]:
all_districts_json.to_csv(r"../data/all-districts-json.csv", index=False)

In [21]:
df[df["district"] == "east"]

,state,district,block,india,1994-post,1994-pre,1994-stations,1995-post,1995-pre,1995-stations,...,2018-post,2018-pre,2018-stations,2019-post,2019-pre,2019-stations,2020-post,2020-pre,2020-stations,total-stations
1214,delhi,east,gandhi nagar,india,NaN,NaN,NaN,5.33,4.40,3.0,...,NaN,NaN,NaN,4.80,4.22,2.0,NaN,NaN,NaN,5.0
1215,delhi,east,preet vihar,india,4.42,4.2,1.0,10.39,10.33,5.0,...,NaN,NaN,NaN,8.87,9.49,2.0,NaN,NaN,NaN,9.0
1216,delhi,east,vivek vihar,india,NaN,NaN,NaN,11.00,11.17,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
